<span style="display:block;text-align:center;margin-right:105px"><img src="../../media/logos/logo-vertical.png" width="200"/></span>

# Section 6: Advanced Simulation Methods

---

## Table of Contents
* [cadCAD Simulation Methods](#cadCAD-Simulation-Methods)
  1. [Monte Carlo Method](#1.-Monte-Carlo-Method)
  2. [Parameter Sweeps](#2.-Parameter-Sweeps)
  3. [A/B Testing](#3.-A/B-Testing)
* [cadCAD Simulation Methods Overview](#cadCAD-Simulation-Methods-Overview)
* [Data Manipulation & Analysis](#Data-Manipulation-&-Analysis)

---

\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(sheep)} \tag{1} \\
\large food_t &\large= food_{t-1} + {\Delta food} \quad \textrm{(tons of grass)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * food_{t-1} - \epsilon * population_{t-1} \quad \textrm{(sheep/month)} \\
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma \quad \textrm{(tons of grass/month)}
\end{align}

where:

$
\begin{align}
\alpha: \quad &\textrm{'reproduction_rate'}\\
\epsilon: \quad &\textrm{'death_rate'}\\
\beta: \quad &\textrm{'consumption_rate'}\\
\gamma: \quad &\textrm{'growth_rate'}\\
\end{align}
$

* A population consumes a food source, and reproduces at a rate proportional to the food source $\alpha$ (alpha), and dies at a rate proportional to the population size $\epsilon$ (epsilon).
* The food source is consumed at a rate proportional to the population $\beta$ (beta), and grows at a constant rate $\gamma$ (gamma).

<center>
<img src="./images/s6-differential-spec-ecosystem-final.png"
     alt="Diff spec"
     style="width: 60%" />
</center>

Import and run the existing ecosystem model code from section 5:

In [ ]:
import math
import pandas as pd
import plotly

pd.options.plotting.backend = "plotly"

from cadCAD.configuration.utils import config_sim

from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [ ]:
from cadCAD.configuration import Experiment

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()

# cadCAD Simulation Methods

1. [Monte Carlo Method](#1.-Monte-Carlo-Method)
2. [Parameter Sweeps](#2.-Parameter-Sweeps)
3. [A/B testing](#3.-A/B-Testing)

## 1. Monte Carlo Method

> Monte Carlo methods, or Monte Carlo experiments, are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. The underlying concept is to use randomness to solve problems that might be deterministic in principle.

Source: https://en.wikipedia.org/wiki/Monte_Carlo_method

### Non-determinism

> In computer science, a nondeterministic algorithm is an algorithm that, even for the same input, can exhibit different behaviors on different runs, as opposed to a deterministic algorithm.

Source: https://en.wikipedia.org/wiki/Nondeterministic_algorithm

\begin{align}
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma * rand() \quad \textrm{(tons of grass/month)} \\
\end{align}


* Addition of a random food growth rate, using `rand()`.

In [ ]:
from numpy import random

In [ ]:
random.rand()

In [ ]:
# Setting a seed so that we can reproduce the experiment
random.seed(1234)

In [ ]:
# With a given seed, generates a random number from a given set
[
    random.rand(),
    random.rand()
]

In [ ]:
random.seed(1234)
[
    random.rand(),
    random.rand()
]

In [ ]:
random.seed(None)

In [ ]:
seeds = [
    random.RandomState(1),
    random.RandomState(2),
    random.RandomState(3),
]

In [ ]:
[
    seeds[0].rand(),
    seeds[1].rand(),
    seeds[2].rand(),
]

In [ ]:
MONTE_CARLO_RUNS = 3

seeds = [random.RandomState(i) for i in range(MONTE_CARLO_RUNS)]
seeds

In [ ]:
initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}

system_params = {
    'reproduction_rate': [0.3],
    'death_rate': [0.03],
    'consumption_rate': [0.03],
    'growth_rate': [30.0],
}

In [ ]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food']
    return 'food', max(food, 0)

In [ ]:
def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_death(params, substep, state_history, previous_state):
    population_death = params['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}

def p_growth(params, substep, state_history, previous_state):
    run = previous_state['run']
    food_growth = params['growth_rate'] * seeds[run - 1].rand()
    return {'delta_food': food_growth}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'death': p_death,
            'consumption': p_consumption,
            'growth': p_growth
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [ ]:
SIMULATION_TIMESTEPS = 500

In [ ]:
sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config,
)

In [ ]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

In [ ]:
df = simulation_result.copy()
df = df[df.simulation == 0]
df

In [ ]:
df[df.run == 1].head()

In [ ]:
df[df.run == 2].head()

In [ ]:
df[df.run == 3].head()

### Simulation Analysis

In [ ]:
import plotly.express as px

px.line(df, x='timestep', y=['population', 'food'], facet_row='run', height=800, width=1000)

## 2. Parameter Sweeps

In [ ]:
dictionary = {
    'reproduction_rate': [0.1, 0.2],
    'death_rate': [0.01, 0.02]
}
dictionary['reproduction_rate'][0]

In [ ]:
nested_dictionary = {
    'population': [
        {'reproduction_rate': 0.1, 'death_rate': 0.01},
        {'reproduction_rate': 0.2, 'death_rate': 0.02},
    ]
}
nested_dictionary['population'][0]['reproduction_rate']

In [ ]:
initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}

system_params = {
    'population': [
        {'reproduction_rate': 0.1, 'death_rate': 0.01},
        {'reproduction_rate': 0.2, 'death_rate': 0.02},
        {'reproduction_rate': 0.3, 'death_rate': 0.03}
    ],
    'food': [
        {'consumption_rate': 0.01, 'growth_rate': 10.0},
        {'consumption_rate': 0.02, 'growth_rate': 20.0},
        {'consumption_rate': 0.03, 'growth_rate': 30.0}
    ]
}

In [ ]:
{
    'population': system_params['population'][0],
    'food': system_params['food'][0]
}

In [ ]:
{
    'population': system_params['population'][1],
    'food': system_params['food'][1]
}

In [ ]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food']
    return 'food', max(food, 0)

In [ ]:
def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['population']['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_death(params, substep, state_history, previous_state):
    population_death = params['population']['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['food']['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}
    
def p_growth(params, substep, state_history, previous_state):
    food_growth = params['food']['growth_rate']
    return {'delta_food': food_growth}

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'death': p_death,
            'consumption': p_consumption,
            'growth': p_growth
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [ ]:
MONTE_CARLO_RUNS = 1
SIMULATION_TIMESTEPS = 500

sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})

In [ ]:
from pprint import pprint

print('system_params')
pprint(system_params)
print(' ')
print('sim_config')
pprint(sim_config)

In [ ]:
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [ ]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

In [ ]:
df = simulation_result.copy()
df = df[df.simulation == 1]
df

### Simulation Analysis

In [ ]:
import plotly.express as px

px.line(df, x='timestep', y=['population', 'food'], facet_row='subset', height=800, width=1000)

## 3. A/B Testing

In [ ]:
df = simulation_result.copy()
df

In [ ]:
import plotly.express as px

fig = px.line(
    df,
    x='timestep',
    y=['population', 'food'],
    facet_row='simulation',
    facet_col='run',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

# cadCAD Simulation Methods Overview

<center>
<img src="./images/cadcad-simulation-methods.png"
     alt="Simulation methods"
     style="width: 600px;" />
</center>

# Data Manipulation & Analysis

In [ ]:
df = simulation_result.copy()
df

In [ ]:
df = df.query("simulation == 1 and run == 2")
df

In [ ]:
fig = px.scatter(
    df[df.timestep >= 80],
    x='population',
    y='food',
    color='timestep'
)
fig.show()

<br/><br/><br/>
# Well done!
You're now well on your way to being a cadCAD system modeller!
<br/><br/><br/><br/>